In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install autokeras

## 2.4 시계열 예측 모델

In [5]:
# 라이브러리 로드
import pandas as pd
import tensorflow as tf
import autokeras as ak

# 판다스 데이터프레임으로 로드
dataset_dir = "/content/drive/MyDrive/Colab Notebooks/data/AirQualityUCI/AirQualityUCI.csv"

dataset = pd.read_csv(dataset_dir, sep=';')

# 데이터 형태 및 컬럼 출력
print(dataset.shape, '\n')
print(dataset.columns)

(9471, 17) 

Index(['Date', 'Time', 'CO(GT)', 'PT08.S1(CO)', 'NMHC(GT)', 'C6H6(GT)',
       'PT08.S2(NMHC)', 'NOx(GT)', 'PT08.S3(NOx)', 'NO2(GT)', 'PT08.S4(NO2)',
       'PT08.S5(O3)', 'T', 'RH', 'AH', 'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')


In [6]:
# 불필요한 컬럼 삭제
dataset = dataset[dataset.columns[:-2]]

# 결측값 NA로 치환
dataset = dataset.dropna()

# 특정 값 치환
dataset = dataset.replace(",", ".", regex=True)

In [7]:
# 검증 데이터셋 분리 (30%)
val_split = int(len(dataset) * 0.7)
data_train = dataset[:val_split]
validation_data = dataset[val_split:]

# 데이터 형태 출력
print(data_train.shape)
print(validation_data.shape)

(6549, 15)
(2808, 15)


In [8]:
 # 데이터 타입 형변환
data_x = data_train[["CO(GT)","PT08.S1(CO)","NMHC(GT)","C6H6(GT)", "PT08.S2(NMHC)","NOx(GT)","PT08.S3(NOx)","NO2(GT)","PT08.S4(NO2)", "PT08.S5(O3)", "T","RH",]].astype("float64")
data_x_val = validation_data[["CO(GT)","PT08.S1(CO)","NMHC(GT)","C6H6(GT)", "PT08.S2(NMHC)","NOx(GT)","PT08.S3(NOx)","NO2(GT)","PT08.S4(NO2)", "PT08.S5(O3)", "T","RH",]].astype("float64")
data_x_test = dataset[["CO(GT)","PT08.S1(CO)","NMHC(GT)","C6H6(GT)", "PT08.S2(NMHC)","NOx(GT)","PT08.S3(NOx)","NO2(GT)","PT08.S4(NO2)", "PT08.S5(O3)", "T","RH",]].astype("float64")

# 정답값 데이터프레임 분리
data_y = data_train["AH"].astype("float64")
data_y_val = validation_data["AH"].astype("float64")

# 데이터 형태 출력
print(data_x.shape)
print(data_y.shape)

(6549, 12)
(6549,)


In [9]:
# 모델 생성
clf = ak.TimeseriesForecaster(
    lookback=3, predict_from=1, predict_until=10,
    max_trials=1, objective="val_loss",
)

# 모델 훈련
clf.fit(
    x=data_x, y=data_y,
    validation_data=(data_x_val, data_y_val),
    batch_size=32, epochs=10,
)

Trial 1 Complete [00h 01m 03s]
val_loss: 2626.331787109375

Best val_loss So Far: 2626.331787109375
Total elapsed time: 00h 01m 03s
Epoch 1/10
205/205 [==============================] - 11s 22ms/step - loss: 899.3455 - mean_squared_error: 899.3455 - val_loss: 3070.1062 - val_mean_squared_error: 3070.1062
Epoch 2/10
205/205 [==============================] - 3s 13ms/step - loss: 884.5486 - mean_squared_error: 884.5486 - val_loss: 2998.1284 - val_mean_squared_error: 2998.1284
Epoch 3/10
205/205 [==============================] - 3s 13ms/step - loss: 860.6165 - mean_squared_error: 860.6165 - val_loss: 2907.4375 - val_mean_squared_error: 2907.4375
Epoch 4/10
205/205 [==============================] - 3s 13ms/step - loss: 833.9654 - mean_squared_error: 833.9654 - val_loss: 2833.3076 - val_mean_squared_error: 2833.3076
Epoch 5/10
205/205 [==============================] - 3s 13ms/step - loss: 815.4286 - mean_squared_error: 815.4286 - val_loss: 2786.2515 - val_mean_squared_error: 2786.2515
Ep

In [10]:
# 모델 예측
predictions = clf.predict(data_x_test)

# 모델 평가
print(clf.evaluate(data_x_val, data_y_val))

88/88 [==============================] - 2s 6ms/step - loss: 2600.2268 - mean_squared_error: 2600.2268
[2600.226806640625, 2600.226806640625]
